### Import

In [ ]:
import math
import numpy as np
import pandas as pd
from collections import defaultdict
from math import floor 
%load_ext cython

### Per-Object Padding (POP)

In [ ]:
%%cython -a
cimport cython
cimport numpy as np
import numpy as np

cdef extern from "math.h":
    double log2(double x) nogil

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def opt_partition_MI(df_sizes, double max_over):
    df_sizes_sorted = df_sizes.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(df_sizes_sorted[2].sum())
    df_sizes_sorted[3] = df_sizes_sorted[2] * total_views
    cdef long num_pages = df_sizes_sorted.shape[0]

    cdef long[::1] s = df_sizes_sorted[1].to_numpy()
    cdef double[::1] p = df_sizes_sorted[3].to_numpy()
    
    cdef double[::1] tracker_best = np.full(num_pages, float('inf'))
    cdef int[::1] tracker_prev = np.empty(num_pages, dtype=np.intc)
    
    cdef int i, j
    cdef double prior_entropy, block_prob, final_entropy, max_size
    
    with nogil:
        for i in range(num_pages):
            if i == 0:
                prior_entropy = 0.
            else:
                prior_entropy = tracker_best[i-1]
        
            block_prob = 0.
            max_size = float(s[i]) * max_over
            for j in range(i,num_pages):
                if float(s[j]) > max_size:
                    break
                block_prob += p[j] 
                final_entropy = block_prob * -log2(block_prob) + prior_entropy
            
                if final_entropy < tracker_best[j]:
                    tracker_best[j] = final_entropy
                    tracker_prev[j] = i

    selected_blocks = []
    current_index = num_pages - 1
    final_entropy = tracker_best[current_index]
    while True:
        b = current_index
        a = tracker_prev[current_index]
        selected_blocks.append((s[a],s[b]))
        
        if a > 0:
            current_index = a - 1
        else:
            selected_blocks.reverse()
            return selected_blocks, final_entropy

### Per-Request Padding (PRP)

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -a
cimport cython
cimport numpy as np
import numpy as np
import pandas as pd
from cython.parallel import prange
from collections import defaultdict

cdef extern from "math.h":
    double log2(double x) nogil

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_j(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_i, double[:] p_ji, double[:] p_j) nogil:
    cdef double j_prob = 0.
    cdef int i
    
    for i in range(j_min[j], j+1):
        j_prob += p_i[i] * p_ji[cndl_idxs[i] + (j-i)]
            
    p_j[j] = j_prob
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_j) nogil:
    cdef double prob_sum = 0.
    cdef int j, i_idx, i_num_cndls, offset
        
    for j in range(i, i_max[i]+1):
        prob_sum += p_j[j]
            
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
            
    for offset in range(i_num_cndls):
        p_ji[i_idx+offset] = p_j[i+offset] / prob_sum
            
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_j(int j, double[:] p_j) nogil:
    cdef double prob = p_j[j]
    
    if prob == 0.:
        return 0.
    else:
        return -prob * log2(prob)
            
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef double log_sum = 0.
    cdef double prob
    cdef int i_idx, idx, i_num_cndls, offset
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        idx = i_idx + offset
        prob = p_ji[idx]
        if prob > 0.:
            log_sum += prob * log2(prob)
        
    return p_i[i] * log_sum


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_max_L(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_ji) nogil:
    cdef double largest_cndl = 0.
    cdef double cndl
    cdef int i
    
    for i in range(j_min[j], j+1):
        cndl = p_ji[cndl_idxs[i] + (j-i)]
        if cndl > largest_cndl:
            largest_cndl = cndl
            
    return largest_cndl 

    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def BlahutArimoto_init(data, double c, long max_itr = 40, double eps = 1e-4):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    cdef int i_num_cndls, i_idx, offset
    cdef double uniform_prob
    
    for i in range(num_sizes):
        i_num_cndls = i_max[i] - i + 1
        i_idx = cndl_idxs[i]
        uniform_prob = 1. / i_num_cndls
        
        for offset in range(i_num_cndls):
            p_ji[i_idx + offset] = uniform_prob
    
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double last_MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, schedule='dynamic', nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    last_MI = p_j_result + p_ji_result
        
    cdef double this_MI
    cdef double max_L = 0.
    cdef long itr
    
    for itr in range(max_itr):
        
        for i in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_ji(i, i_max, cndl_idxs, p_ji, p_j)  
            
        for j in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
        
        if itr % 10 == 0:
            p_j_result = 0.
            p_ji_result = 0.
        
            for j in prange(num_sizes, nogil=True):
                p_j_result += calc_p_j(j, p_j)
        
            for i in prange(num_sizes, schedule='dynamic', nogil=True):
                p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
            this_MI = p_j_result + p_ji_result
        
            if this_MI <= last_MI and last_MI - this_MI < eps:    
                break
            
            last_MI = this_MI
            
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        max_L += calc_max_L(j, j_min, cndl_idxs, p_ji)
        
    p_j_dict = defaultdict(float)
    size_to_i = {}
    
    for j in range(num_sizes):
        p_j_dict[s[j]] = p_j[j]
        size_to_i[s[j]] = j    
    
    return i_max, cndl_idxs, p_ji, p_i, s, p_j_dict, size_to_i, this_MI, max_L


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def BlahutArimoto_incr(data, double c, p_j_dict, long max_itr = 40, double eps = 1e-4):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    for j in range(num_sizes):
        p_j[j] = p_j_dict[s[j]]
    
    cdef int i_num_cndls, i_idx, offset
    
    for i in prange(num_sizes, schedule='dynamic', nogil=True):
        update_p_ji(i, i_max, cndl_idxs, p_ji, p_j) 
    
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double last_MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, schedule='dynamic', nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    last_MI = p_j_result + p_ji_result
        
    cdef double this_MI
    cdef double max_L = 0.
    cdef long itr
    
    for itr in range(max_itr):
        
        for i in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_ji(i, i_max, cndl_idxs, p_ji, p_j)  
            
        for j in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
        
        if itr % 10 == 0:
            p_j_result = 0.
            p_ji_result = 0.
        
            for j in prange(num_sizes, nogil=True):
                p_j_result += calc_p_j(j, p_j)
        
            for i in prange(num_sizes, schedule='dynamic', nogil=True):
                p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
            this_MI = p_j_result + p_ji_result
        
            if this_MI <= last_MI and last_MI - this_MI < eps:    
                break
            
            last_MI = this_MI
            
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        max_L += calc_max_L(j, j_min, cndl_idxs, p_ji)
        
    p_j_dict = defaultdict(float)
    size_to_i = {}
    
    for j in range(num_sizes):
        p_j_dict[s[j]] = p_j[j]
        size_to_i[s[j]] = j    
    
    return i_max, cndl_idxs, p_ji, p_i, s, p_j_dict, size_to_i, this_MI, max_L

### Padding without a Distribution (PwoD)

In [ ]:
def pureMaxL(df_sizes, max_over):
    u_sizes = sorted(df_sizes[1].unique().tolist(), reverse=True)
    
    size_map = {}
    
    cur_ceil = u_sizes[0]
    cur_floor = cur_ceil / max_over
    
    for size in u_sizes:
        if size < cur_floor:
            cur_ceil = size
            cur_floor = cur_ceil / max_over
        size_map[size] = cur_ceil
        
    return size_map

### Padme
K. Nikitin, L. Barman, W. Lueks, M. Underwood, J.-P. Hubaux, and B. Ford. "Reducing metadata leakage from encrypted files and communications with PURBs," _Proceedings on Privacy Enhancing Technologies_, vol. 2019, no. 4, pp. 6-33, 2019.

Available at: https://petsymposium.org/2019/files/papers/issue4/popets-2019-0056.pdf

This code is from: https://github.com/dedis/purb

In [ ]:
def log(x):
    return math.log(x, 2)

def getPadme(L):
    L = int(L)
    U = int(floor(log(L)))
    V = int(floor(log(U))+1)
    lastBits = U-V

    bitMask = (2 ** lastBits - 1)

    if L & bitMask == 0:
        return L

    L += (1 << lastBits)
    L = ((L >> lastBits) << lastBits)

    return L

### P-ALPaCA
G. Cherubin, J. Hayes, and M. Juarez, "Website fingerprinting defenses at the application layer," _Proceedings on Privacy Enhancing Technologies_, vol. 2017, no. 2, pp. 186-203, 2017.

Available at: https://www.petsymposium.org/2017/papers/issue2/paper54-2017-2-source.pdf

This code is our implementation of P-ALPaCA, as applied to our setting.

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -a
cimport cython
cimport numpy as np
import numpy as np
from cython.parallel import prange
from collections import defaultdict


cdef extern from "math.h":
    double log2(double x) nogil

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void init_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef int j, i_num_cndls, i_idx, offset
    cdef double block_prob
    
    block_prob = 0.
        
    for j in range(i, i_max[i]+1):
        block_prob += p_i[j]
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        p_ji[i_idx + offset] = p_i[i + offset] / block_prob
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_j(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_i, double[:] p_ji, double[:] p_j) nogil:
    cdef double j_prob = 0.
    cdef int i
    
    for i in range(j_min[j], j+1):
        j_prob += p_i[i] * p_ji[cndl_idxs[i] + (j-i)]
            
    p_j[j] = j_prob    
            
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_j(int j, double[:] p_j) nogil:
    cdef double prob = p_j[j]
    
    if prob == 0.:
        return 0.
    else:
        return -prob * log2(prob)
            
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef double log_sum = 0.
    cdef double prob
    cdef int i_idx, idx, i_num_cndls, offset
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        idx = i_idx + offset
        prob = p_ji[idx]
        if prob > 0.:
            log_sum += prob * log2(prob)
        
    return p_i[i] * log_sum


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_max_L(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_ji) nogil:
    cdef double largest_cndl = 0.
    cdef double cndl
    cdef int i
    
    for i in range(j_min[j], j+1):
        cndl = p_ji[cndl_idxs[i] + (j-i)]
        if cndl > largest_cndl:
            largest_cndl = cndl
            
    return largest_cndl 


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def p_alpaca_return_all(data, double c):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    for i in prange(num_sizes, nogil=True):
        init_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
    
    for j in prange(num_sizes, nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    MI = p_j_result + p_ji_result
    
    cdef double max_L = 0.
    
    for j in prange(num_sizes, nogil=True):
        max_L += calc_max_L(j, j_min, cndl_idxs, p_ji)

    p_j_dict = defaultdict(float)
    size_to_i = {}
    
    for j in range(num_sizes):
        p_j_dict[s[j]] = p_j[j]
        size_to_i[s[j]] = j    
    
    return i_max, cndl_idxs, p_ji, p_i, s, p_j_dict, size_to_i, MI, max_L

### D-ALPaCA
G. Cherubin, J. Hayes, and M. Juarez, "Website fingerprinting defenses at the application layer," _Proceedings on Privacy Enhancing Technologies_, vol. 2017, no. 2, pp. 186-203, 2017.

Available at: https://www.petsymposium.org/2017/papers/issue2/paper54-2017-2-source.pdf

This code is our implementation of D-ALPaCA, as applied to our setting.

In [ ]:
def getDALPaCA(size, bin_size):
    if bin_size == 0:
        return size
    if size % bin_size == 0:
        return size
    return size + (bin_size - (size % bin_size))